# Capstone week 3 part3
## Analysis of data, segmentation, clustering

In [2]:
import pandas as pd
pc_locs_df = pd.read_csv('loc1.csv').drop('Unnamed: 0', axis=1)
pc_locs_df.head()

,PostalCode,Borough,Neighborhood,latitude,longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Regent Park / Harbourfront,43.654260,-79.360636
3,M6A,North York,Lawrence Manor / Lawrence Heights,43.718518,-79.464763
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government,43.662301,-79.389494


### Drop postal code column

In [3]:
pc_locs_df = pc_locs_df.drop('PostalCode', axis=1)

### getting marked map function

In [7]:
import folium

def get_marked_map(data, latitude, longitude):
    map_borough = folium.Map(location=[latitude, longitude], zoom_start=11)

    # add markers to map
    for lat, lng, label in zip(data['latitude'], data['longitude'], data['Neighborhood']):
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(map_borough)
    return map_borough

## Marked map with all areas in Toronto

In [8]:
from geopy.geocoders import Nominatim

# get cords of Toronto
address = 'Toronto, Canada'
geolocator = Nominatim(user_agent="canada_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

# create map of canada using latitude and longitude values
map_newyork = get_marked_map(pc_locs_df, latitude, longitude)
map_newyork

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


### Function for getting cords using address

In [10]:
def get_lat_lon(address):
    location = geolocator.geocode(address)
    latitude = location.latitude
    longitude = location.longitude
    return latitude, longitude

## Listing different areas in Toronto

In [11]:
set(pc_locs_df['Borough'].values)

{'Central Toronto',
 'Downtown Toronto',
 'East Toronto',
 'East York',
 'Etobicoke',
 'Mississauga',
 'North York',
 'Scarborough',
 'West Toronto',
 'York'}

### Doing analysis only for Central Toronto

In [12]:
def borough_data(area):
    return pc_locs_df[pc_locs_df['Borough'] == 'Central Toronto'].reset_index(drop=True)

## Get data related to central toronto

In [13]:
borough_data =  borough_data('Central Toronto')
borough_data.head()

,Borough,Neighborhood,latitude,longitude
0,Central Toronto,Lawrence Park,43.728020,-79.388790
1,Central Toronto,Roselawn,43.711695,-79.416936
2,Central Toronto,Davisville North,43.712751,-79.390197
3,Central Toronto,Forest Hill North &amp; West,43.696948,-79.411307
4,Central Toronto,North Toronto West,43.715383,-79.405678


## Get cords of central toronto

In [14]:
borough_address = '{}, Toronto'.format('Central Toronto')
latitude, longitude = get_lat_lon(borough_address)
print('The geograpical coordinate of {} are {}, {}.'.format(borough, latitude, longitude))

NameError: name 'borough' is not defined

In [15]:
map_borough = folium.Map(location=[latitude, longitude], zoom_start=11)
map_borough = get_marked_map(borough_data, latitude, longitude)
map_borough

Next, we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them.
### Define Foursquare Credentials and Version

In [16]:
CLIENT_ID = 'xxxxxxxxxxxxxxxxx' # your Foursquare ID
CLIENT_SECRET = 'xxxxxxxxxxxxxxxxx' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 100

In [17]:
borough_data.loc[0, 'Neighborhood']

'Lawrence Park'

In [18]:
neighborhood_latitude = borough_data.loc[0, 'latitude'] # neighborhood latitude value
neighborhood_longitude = borough_data.loc[0, 'longitude'] # neighborhood longitude value

neighborhood_name = borough_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Lawrence Park are 43.7280205, -79.3887901.


## getting top 100 places in 500m radius

In [19]:
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=RJF3FAXJURKJO53ZRAV2JEDN2UFJLGC3D11P0VX4KUJG1FUY&client_secret=MJAWMP1RQCARV5QSHZSSWCTFMYMRATSYAW4IA0SS4GTCTZ30&v=20180604&ll=43.7280205,-79.3887901&radius=500&limit=100'

In [21]:
import requests

results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ea57c2a211536324bf73ba1'},
 'response': {'headerLocation': 'Toronto',
  'headerFullLocation': 'Toronto',
  'headerLocationGranularity': 'city',
  'totalResults': 4,
  'suggestedBounds': {'ne': {'lat': 43.7325205045, 'lng': -79.3825744605273},
   'sw': {'lat': 43.7235204955, 'lng': -79.3950057394727}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '50e6da19e4b0d8a78a0e9794',
       'name': 'Lawrence Park Ravine',
       'location': {'address': '3055 Yonge Street',
        'crossStreet': 'Lawrence Avenue East',
        'lat': 43.72696303913755,
        'lng': -79.39438246708775,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.72696303913755,
          'lng': -79.39438246708775}],
        'distance': 465,
        'c

In [22]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [26]:
from pandas.io.json import json_normalize

venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Lawrence Park Ravine,Park,43.726963,-79.394382
1,Dim Sum Deluxe,Dim Sum Restaurant,43.726953,-79.394260
2,Zodiac Swim School,Swim School,43.728532,-79.382860
3,TTC Bus #162 - Lawrence-Donway,Bus Line,43.728026,-79.382805


In [27]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

4 venues were returned by Foursquare.


## 2. Explore Neighborhoods in Central Toronto

In [28]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [29]:
borough_venues = getNearbyVenues(names=borough_data['Neighborhood'],
                                   latitudes=borough_data['latitude'],
                                   longitudes=borough_data['longitude']
                                  )

Lawrence Park
Roselawn
Davisville North
Forest Hill North &amp; West
North Toronto West
The Annex / North Midtown / Yorkville
Davisville
Moore Park / Summerhill East
Summerhill West / Rathnelly / South Hill / Forest Hill SE / Deer Park


In [30]:
print(borough_venues.shape)
borough_venues.head()

(119, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Lawrence Park,43.728020,-79.388790,Lawrence Park Ravine,43.726963,-79.394382,Park
1,Lawrence Park,43.728020,-79.388790,Dim Sum Deluxe,43.726953,-79.394260,Dim Sum Restaurant
2,Lawrence Park,43.728020,-79.388790,Zodiac Swim School,43.728532,-79.382860,Swim School
3,Lawrence Park,43.728020,-79.388790,TTC Bus #162 - Lawrence-Donway,43.728026,-79.382805,Bus Line
4,Roselawn,43.711695,-79.416936,Rosalind's Garden Oasis,43.712189,-79.411978,Garden


In [31]:
borough_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Davisville,39,39,39,39,39,39
Davisville North,7,7,7,7,7,7
Forest Hill North &amp; West,4,4,4,4,4,4
Lawrence Park,4,4,4,4,4,4
Moore Park / Summerhill East,3,3,3,3,3,3
North Toronto West,21,21,21,21,21,21
Roselawn,2,2,2,2,2,2
Summerhill West / Rathnelly / South Hill / Forest Hill SE / Deer Park,16,16,16,16,16,16
The Annex / North Midtown / Yorkville,23,23,23,23,23,23


In [32]:
print('There are {} uniques categories.'.format(len(borough_venues['Venue Category'].unique())))

There are 68 uniques categories.


## 3. Analyze Each Neighborhood

In [33]:
# one hot encoding
borough_onehot = pd.get_dummies(borough_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
borough_onehot['Neighborhood'] = borough_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [borough_onehot.columns[-1]] + list(borough_onehot.columns[:-1])
borough_onehot = borough_onehot[fixed_columns]

borough_onehot.head()

,Neighborhood,American Restaurant,Asian Restaurant,BBQ Joint,Bagel Shop,Bank,Bar,Breakfast Spot,Brewery,Burger Joint,...,Sports Bar,Supermarket,Sushi Restaurant,Swim School,Thai Restaurant,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Yoga Studio
0,Lawrence Park,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Lawrence Park,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Lawrence Park,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
3,Lawrence Park,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Roselawn,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [34]:
borough_onehot.shape

(119, 69)

In [35]:
borough_grouped = borough_onehot.groupby('Neighborhood').mean().reset_index()
borough_grouped

,Neighborhood,American Restaurant,Asian Restaurant,BBQ Joint,Bagel Shop,Bank,Bar,Breakfast Spot,Brewery,Burger Joint,...,Sports Bar,Supermarket,Sushi Restaurant,Swim School,Thai Restaurant,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Yoga Studio
0,Davisville,0.000000,0.025641,0.000000,0.000000,0.0000,0.025641,0.000000,0.025641,0.000000,...,0.0000,0.0000,0.051282,0.00,0.025641,0.025641,0.000000,0.000000,0.0000,0.000000
1,Davisville North,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.142857,0.000000,0.000000,...,0.0000,0.0000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000
2,Forest Hill North &amp; West,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,...,0.0000,0.0000,0.250000,0.00,0.000000,0.000000,0.250000,0.000000,0.0000,0.000000
3,Lawrence Park,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,...,0.0000,0.0000,0.000000,0.25,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000
4,Moore Park / Summerhill East,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,...,0.0000,0.0000,0.000000,0.00,0.000000,0.000000,0.333333,0.000000,0.0000,0.000000
5,North Toronto West,0.000000,0.000000,0.000000,0.047619,0.0000,0.000000,0.000000,0.000000,0.000000,...,0.0000,0.0000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0000,0.047619
6,Roselawn,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,...,0.0000,0.0000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000
7,Summerhill West / Rathnelly / South Hill / For...,0.062500,0.000000,0.000000,0.062500,0.0625,0.000000,0.000000,0.000000,0.000000,...,0.0625,0.0625,0.062500,0.00,0.000000,0.000000,0.000000,0.000000,0.0625,0.000000
8,The Annex / North Midtown / Yorkville,0.043478,0.000000,0.043478,0.000000,0.0000,0.000000,0.000000,0.000000,0.043478,...,0.0000,0.0000,0.000000,0.00,0.000000,0.000000,0.000000,0.043478,0.0000,0.000000


In [36]:
borough_grouped.shape

(9, 69)

In [37]:
num_top_venues = 5

for hood in borough_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = borough_grouped[borough_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Davisville----
            venue  freq
0    Dessert Shop  0.08
1  Sandwich Place  0.08
2     Pizza Place  0.08
3             Gym  0.05
4     Coffee Shop  0.05


----Davisville North----
               venue  freq
0   Department Store  0.14
1     Breakfast Spot  0.14
2                Gym  0.14
3               Park  0.14
4  Food & Drink Shop  0.14


----Forest Hill North &amp; West----
                venue  freq
0  Mexican Restaurant  0.25
1               Trail  0.25
2       Jewelry Store  0.25
3    Sushi Restaurant  0.25
4         Music Venue  0.00


----Lawrence Park----
                venue  freq
0            Bus Line  0.25
1         Swim School  0.25
2                Park  0.25
3  Dim Sum Restaurant  0.25
4  Light Rail Station  0.00


----Moore Park / Summerhill East----
                       venue  freq
0                 Playground  0.33
1                      Trail  0.33
2                       Park  0.33
3        American Restaurant  0.00
4  Middle Eastern Restaurant  0.00


In [38]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [40]:
import numpy as np

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = borough_grouped['Neighborhood']

for ind in np.arange(borough_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(borough_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Davisville,Sandwich Place,Dessert Shop,Pizza Place,Sushi Restaurant,Coffee Shop,Café,Italian Restaurant,Gym,Gas Station,Bar
1,Davisville North,Gym,Food & Drink Shop,Sandwich Place,Park,Breakfast Spot,Department Store,Hotel,Fried Chicken Joint,Furniture / Home Store,Flower Shop
2,Forest Hill North &amp; West,Jewelry Store,Trail,Sushi Restaurant,Mexican Restaurant,Farmers Market,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Donut Shop
3,Lawrence Park,Dim Sum Restaurant,Park,Swim School,Bus Line,Yoga Studio,Farmers Market,Dessert Shop,Diner,Donut Shop,Fast Food Restaurant
4,Moore Park / Summerhill East,Playground,Trail,Park,Yoga Studio,Donut Shop,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Farmers Market


### 4. Cluster Neighborhoods

In [42]:
from sklearn.cluster import KMeans

# set number of clusters
kclusters = 5

borough_grouped_clustering = borough_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(borough_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 3, 4, 2, 0, 1, 0, 0], dtype=int32)

In [43]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

borough_merged = borough_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
borough_merged = borough_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

borough_merged.head() # check the last columns!

,Borough,Neighborhood,latitude,longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Central Toronto,Lawrence Park,43.728020,-79.388790,4,Dim Sum Restaurant,Park,Swim School,Bus Line,Yoga Studio,Farmers Market,Dessert Shop,Diner,Donut Shop,Fast Food Restaurant
1,Central Toronto,Roselawn,43.711695,-79.416936,1,Garden,Music Venue,Yoga Studio,Farmers Market,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Donut Shop,Fast Food Restaurant
2,Central Toronto,Davisville North,43.712751,-79.390197,0,Gym,Food & Drink Shop,Sandwich Place,Park,Breakfast Spot,Department Store,Hotel,Fried Chicken Joint,Furniture / Home Store,Flower Shop
3,Central Toronto,Forest Hill North &amp; West,43.696948,-79.411307,3,Jewelry Store,Trail,Sushi Restaurant,Mexican Restaurant,Farmers Market,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Donut Shop
4,Central Toronto,North Toronto West,43.715383,-79.405678,0,Clothing Store,Coffee Shop,Yoga Studio,Café,Fast Food Restaurant,Diner,Dessert Shop,Mexican Restaurant,Park,Restaurant


In [46]:
import matplotlib.cm as cm
import matplotlib.colors as colors

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(
    borough_merged['latitude'], borough_merged['longitude'], borough_merged['Neighborhood'], borough_merged['Cluster Labels']
):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## 5. Examine Clusters

In [47]:
def examine(label):
    return borough_merged.loc[
        borough_merged['Cluster Labels'] == 0,
        borough_merged.columns[
            [1] + list(range(5, borough_merged.shape[1]))
        ]
    ]

In [48]:
examine(0)

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Davisville North,Gym,Food & Drink Shop,Sandwich Place,Park,Breakfast Spot,Department Store,Hotel,Fried Chicken Joint,Furniture / Home Store,Flower Shop
4,North Toronto West,Clothing Store,Coffee Shop,Yoga Studio,Café,Fast Food Restaurant,Diner,Dessert Shop,Mexican Restaurant,Park,Restaurant
5,The Annex / North Midtown / Yorkville,Sandwich Place,Café,Coffee Shop,American Restaurant,Donut Shop,History Museum,Indian Restaurant,Liquor Store,Middle Eastern Restaurant,Park
6,Davisville,Sandwich Place,Dessert Shop,Pizza Place,Sushi Restaurant,Coffee Shop,Café,Italian Restaurant,Gym,Gas Station,Bar
8,Summerhill West / Rathnelly / South Hill / For...,Pub,Coffee Shop,Sports Bar,Vietnamese Restaurant,Light Rail Station,Liquor Store,Pizza Place,Restaurant,American Restaurant,Supermarket


In [49]:
examine(1)

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Davisville North,Gym,Food & Drink Shop,Sandwich Place,Park,Breakfast Spot,Department Store,Hotel,Fried Chicken Joint,Furniture / Home Store,Flower Shop
4,North Toronto West,Clothing Store,Coffee Shop,Yoga Studio,Café,Fast Food Restaurant,Diner,Dessert Shop,Mexican Restaurant,Park,Restaurant
5,The Annex / North Midtown / Yorkville,Sandwich Place,Café,Coffee Shop,American Restaurant,Donut Shop,History Museum,Indian Restaurant,Liquor Store,Middle Eastern Restaurant,Park
6,Davisville,Sandwich Place,Dessert Shop,Pizza Place,Sushi Restaurant,Coffee Shop,Café,Italian Restaurant,Gym,Gas Station,Bar
8,Summerhill West / Rathnelly / South Hill / For...,Pub,Coffee Shop,Sports Bar,Vietnamese Restaurant,Light Rail Station,Liquor Store,Pizza Place,Restaurant,American Restaurant,Supermarket


In [50]:
examine(2)

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Davisville North,Gym,Food & Drink Shop,Sandwich Place,Park,Breakfast Spot,Department Store,Hotel,Fried Chicken Joint,Furniture / Home Store,Flower Shop
4,North Toronto West,Clothing Store,Coffee Shop,Yoga Studio,Café,Fast Food Restaurant,Diner,Dessert Shop,Mexican Restaurant,Park,Restaurant
5,The Annex / North Midtown / Yorkville,Sandwich Place,Café,Coffee Shop,American Restaurant,Donut Shop,History Museum,Indian Restaurant,Liquor Store,Middle Eastern Restaurant,Park
6,Davisville,Sandwich Place,Dessert Shop,Pizza Place,Sushi Restaurant,Coffee Shop,Café,Italian Restaurant,Gym,Gas Station,Bar
8,Summerhill West / Rathnelly / South Hill / For...,Pub,Coffee Shop,Sports Bar,Vietnamese Restaurant,Light Rail Station,Liquor Store,Pizza Place,Restaurant,American Restaurant,Supermarket


In [51]:
examine(3)

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Davisville North,Gym,Food & Drink Shop,Sandwich Place,Park,Breakfast Spot,Department Store,Hotel,Fried Chicken Joint,Furniture / Home Store,Flower Shop
4,North Toronto West,Clothing Store,Coffee Shop,Yoga Studio,Café,Fast Food Restaurant,Diner,Dessert Shop,Mexican Restaurant,Park,Restaurant
5,The Annex / North Midtown / Yorkville,Sandwich Place,Café,Coffee Shop,American Restaurant,Donut Shop,History Museum,Indian Restaurant,Liquor Store,Middle Eastern Restaurant,Park
6,Davisville,Sandwich Place,Dessert Shop,Pizza Place,Sushi Restaurant,Coffee Shop,Café,Italian Restaurant,Gym,Gas Station,Bar
8,Summerhill West / Rathnelly / South Hill / For...,Pub,Coffee Shop,Sports Bar,Vietnamese Restaurant,Light Rail Station,Liquor Store,Pizza Place,Restaurant,American Restaurant,Supermarket


In [52]:
examine(4)

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Davisville North,Gym,Food & Drink Shop,Sandwich Place,Park,Breakfast Spot,Department Store,Hotel,Fried Chicken Joint,Furniture / Home Store,Flower Shop
4,North Toronto West,Clothing Store,Coffee Shop,Yoga Studio,Café,Fast Food Restaurant,Diner,Dessert Shop,Mexican Restaurant,Park,Restaurant
5,The Annex / North Midtown / Yorkville,Sandwich Place,Café,Coffee Shop,American Restaurant,Donut Shop,History Museum,Indian Restaurant,Liquor Store,Middle Eastern Restaurant,Park
6,Davisville,Sandwich Place,Dessert Shop,Pizza Place,Sushi Restaurant,Coffee Shop,Café,Italian Restaurant,Gym,Gas Station,Bar
8,Summerhill West / Rathnelly / South Hill / For...,Pub,Coffee Shop,Sports Bar,Vietnamese Restaurant,Light Rail Station,Liquor Store,Pizza Place,Restaurant,American Restaurant,Supermarket
